In [1]:
# import cell
import datetime

import QuantLib as ql
import plotly.express as px
from pandas import DataFrame
from jupyterUtils import export, exportPlotlyFigure
from marketdata import init_marketdata
from marketdata.init_marketdata import today
from marketdata.interestRateCurves import LiborCurve
from quantlibpythoncookbook_extras import utils
asdf =1

However, the approach of SA-CCR to use the Black-Scholes for interest rate derivatives does have a significant flaw.

Obviously, $\ln\left(P_i / K_i \right)$ is not defined, if $P_i/K_i$ is negative. This has, however been very commonplace in recent years, especially for Euro swaptions. One can always set the fixed rate of the underlying swap and therefore $K_i$ as a negative value. But based on the market data of the 10th of May 2019 it is even possible to yield negative par rates for the underlying swap.

As an example we are plotting the zero and the 6M forward curve for the EURIBOR 6M index below.

In [2]:
euribor_curve = LiborCurve.EURIBOR6M.value

end= today+ql.Period(30, ql.Years)
dates= [ql.Date(serial) for serial in range((today+ql.Period(4, ql.Days)).serialNumber(), end.serialNumber()+1)]
forward_rates = [euribor_curve.forwardRate(d, ql.TARGET().advance(d,6,ql.Months), ql.Actual360(), ql.Simple).rate() for d in dates]
zero_rates = [euribor_curve.zeroRate(d, ql.Actual360(), ql.Simple).rate() for d in dates]

In [3]:
# _, ax = utils.plot()
# utils.highlight_x_axis(ax)
# utils.plot_curve(ax, dates, [(forward_rates, '-')], format_rates=True)
# utils.plot_curve(ax, dates, [(zero_rates, '.')], format_rates=True)

In [4]:
dates2 = []
for date in dates:
    dates2.append(datetime.datetime.strptime('%02d-%02d-%d' % (date.dayOfMonth(), date.month(), date.year()),'%d-%m-%Y'))

#Trying to plot with plotly instead of matplotlib
df1 = DataFrame(list(zip(dates2, forward_rates)), columns = ['date', 'rate'])
df1['CurveType'] = '6M Forward Rate'
df2 = DataFrame(list(zip(dates2, zero_rates)), columns = ['date', 'rate'])
df2['CurveType'] = 'Zero Rate'
df3 = df1.append(df2)

In [6]:
fig = px.line(df3, x='date', y='rate', color='CurveType')
fig.update_layout(legend_orientation="h")

exportPlotlyFigure(fig, name='euribor6m_termstructure')

In [7]:
export('issue_regulatory_swaption_delta.ipynb')